# Censo Escolar da Educação Básica

## Configuração do ambiente

In [0]:
spark.conf.set(
  "fs.azure.account.key.m03storage.blob.core.windows.net",
  "3Ts0JpdSfDvYx8JOerU9CrExx1Ankj3W1q5J3Cscu2hXlQR90cpMlVy6p2mBB0NpKndKWixIcp1poqcFrAJ+oQ==")

In [0]:
blobPath = 'wasbs://m03container@m03storage.blob.core.windows.net/'
bronzePath = 'mnt/bronze/censo_escolar/'
silverPath = 'mnt/silver/censo_escolar/'
goldPath = 'mnt/gold/censo_escolar/'

Out[2]: [FileInfo(path='wasbs://m03container@m03storage.blob.core.windows.net/censo_escolar_2017.zip', name='censo_escolar_2017.zip', size=1750578112),
 FileInfo(path='wasbs://m03container@m03storage.blob.core.windows.net/censo_escolar_2018.zip', name='censo_escolar_2018.zip', size=2692342575),
 FileInfo(path='wasbs://m03container@m03storage.blob.core.windows.net/censo_escolar_2019.zip', name='censo_escolar_2019.zip', size=2804740605),
 FileInfo(path='wasbs://m03container@m03storage.blob.core.windows.net/mnt/', name='mnt/', size=0)]

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Download das Bases de Dados

In [0]:
%sh
echo 'Iniciando download do Censo Escolar 2017...'
wget https://download.inep.gov.br/microdados/micro_censo_escolar_2017.zip -O /dbfs/mnt/projetom03/censo_escolar_2017.zip

Iniciando download do Censo Escolar 2017...
--2021-03-25 15:04:42-- https://download.inep.gov.br/microdados/micro_censo_escolar_2017.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1750578112 (1.6G) [application/zip]
Saving to: ‘/dbfs/mnt/projetom03/censo_escolar_2017.zip’

 0K .......... .......... .......... .......... .......... 0% 1.15M 24m15s
 50K .......... .......... .......... .......... .......... 0% 2.17M 18m32s
 100K .......... .......... .......... .......... .......... 0% 2.25M 16m28s
 150K .......... .......... .......... .......... .......... 0% 2.32M 15m21s
 200K .......... .......... .......... .......... .......... 0% 12.9M 12m42s
 250K .......... .......... .......... .......... .......... 0% 2.76M 12m16s
 300K .......... .......... .......... .......... .......... 0% 2.88M 11m54s
 350K .......... .......... .......... .......... .......... 0% 8.42M 10m49s
 400K .......... .......... .......... .......... .......... 0% 9.64M 9m56s
 450K .......... .......... .......... .......... .......... 0% 520K 14m25s
 500K .......... .......... .......... .......... .......... 0% 552K 17m48s
 550K .......... .......... .......... .......... .......... 0% 10.2M 16m33s
 600K .......... .......... .......... .......... .......... 0% 19.4M 15m23s
 650K .......... .......... .......... .......... .......... 0% 2.99M 14m57s
 700K .......... .......... .......... .......... .......... 0% 8.70M 14m10s
 750K .......... .......... .......... .......... .......... 0% 3.14M 13m50s
 800K .......... .......... .......... .......... .......... 0% 11.0M 13m10s
 850K .......... .......... .......... .......... .......... 0% 15.2M 12m32s
 900K .......... .......... .......... .......... .......... 0% 2.47M 12m28s
 950K .......... .......... .......... .......... .......... 0% 12.6M 11m57s
 1000K .......... .......... .......... .......... .......... 0% 2.95M 11m50s
 1050K .......... .......... .......... .......... .......... 0% 16.5M 11m22s
 1100K .......... .......... .......... .......... .......... 0% 21.7M 10m56s
 1150K .......... .......... .......... .......... .......... 0% 1.86M 11m6s
 1200K .......... .......... .......... .......... .......... 0% 10.4M 10m46s
 1250K .......... .......... .......... .......... .......... 0% 4.09M 10m37s
 1300K .......... .......... .......... .......... .......... 0% 13.4M 10m18s
 1350K .......... .......... .......... .......... .......... 0% 2.95M 10m16s
 1400K .......... .......... .......... .......... .......... 0% 15.9M 9m58s
 1450K .......... .......... .......... .......... .......... 0% 2.75M 9m58s
 1500K .......... .......... .......... .......... .......... 0% 11.8M 9m44s
 1550K .......... .......... .......... .......... .......... 0% 12.0M 9m30s
 1600K .......... .......... .......... .......... .......... 0% 2.81M 9m30s
 1650K .......... .......... .......... .......... .......... 0% 4.92M 9m24s
 1700K .......... .......... .......... .......... .......... 0% 4.71M 9m18s
 1750K .......... .......... .......... .......... .......... 0% 13.5M 9m6s
 1800K .......... .......... .......... .......... .......... 0% 2.92M 9m6s
 1850K .......... .......... .......... .......... .......... 0% 8.90M 8m57s
 1900K .......... .......... .......... .......... .......... 0% 15.6M 8m46s
 1950K .......... .......... .......... .......... .......... 0% 2.39M 8m50s
 2000K .......... .......... .......... .......... .......... 0% 19.3M 8m39s
 2050K .......... .......... .......... .......... .......... 0% 3.22M 8m39s
 2100K .......... .......... .......... .......... .......... 0% 11.5M 8m30s
 2150K .......... .......... .......... .......... .......... 0% 17.0M 8m21s
 2200K .......... .......... .......... .......... .......... 0% 2.83M 8m23s
 2250K .......... .......... .......... .......... .......... 0% 10.3M 8m16s
 2300K ....

In [0]:
%sh
echo 'Iniciando download do Censo Escolar 2018...'
wget https://download.inep.gov.br/microdados/microdados_educacao_basica_2018.zip -O /dbfs/mnt/projetom03/censo_escolar_2018.zip

Iniciando download do Censo Escolar 2018...
--2021-03-25 14:56:44-- https://download.inep.gov.br/microdados/microdados_educacao_basica_2018.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692342575 (2.5G) [application/zip]
Saving to: ‘/dbfs/mnt/projetom03/censo_escolar_2018.zip’

 0K .......... .......... .......... .......... .......... 0% 1.21M 35m26s
 50K .......... .......... .......... .......... .......... 0% 2.16M 27m37s
 100K .......... .......... .......... .......... .......... 0% 2.50M 24m7s
 150K .......... .......... .......... .......... .......... 0% 2.73M 22m0s
 200K .......... .......... .......... .......... .......... 0% 15.6M 18m9s
 250K .......... .......... .......... .......... .......... 0% 2.71M 17m45s
 300K .......... .......... .......... .......... .......... 0% 14.5M 15m38s
 350K .......... .......... .......... .......... .......... 0% 3.15M 15m23s
 400K .......... .......... .......... .......... .......... 0% 12.4M 14m3s
 450K .......... .......... .......... .......... .......... 0% 3.21M 13m59s
 500K .......... .......... .......... .......... .......... 0% 17.3M 12m56s
 550K .......... .......... .......... .......... .......... 0% 16.3M 12m5s
 600K .......... .......... .......... .......... .......... 0% 2.83M 12m19s
 650K .......... .......... .......... .......... .......... 0% 14.3M 11m39s
 700K .......... .......... .......... .......... .......... 0% 3.21M 11m45s
 750K .......... .......... .......... .......... .......... 0% 13.6M 11m13s
 800K .......... .......... .......... .......... .......... 0% 15.5M 10m43s
 850K .......... .......... .......... .......... .......... 0% 3.05M 10m54s
 900K .......... .......... .......... .......... .......... 0% 18.0M 10m27s
 950K .......... .......... .......... .......... .......... 0% 2.98M 10m39s
 1000K .......... .......... .......... .......... .......... 0% 11.9M 10m19s
 1050K .......... .......... .......... .......... .......... 0% 23.0M 9m56s
 1100K .......... .......... .......... .......... .......... 0% 3.06M 10m6s
 1150K .......... .......... .......... .......... .......... 0% 13.4M 9m49s
 1200K .......... .......... .......... .......... .......... 0% 2.99M 10m0s
 1250K .......... .......... .......... .......... .......... 0% 13.4M 9m44s
 1300K .......... .......... .......... .......... .......... 0% 16.1M 9m28s
 1350K .......... .......... .......... .......... .......... 0% 3.12M 9m37s
 1400K .......... .......... .......... .......... .......... 0% 16.1M 9m23s
 1450K .......... .......... .......... .......... .......... 0% 3.05M 9m32s
 1500K .......... .......... .......... .......... .......... 0% 13.7M 9m20s
 1550K .......... .......... .......... .......... .......... 0% 15.6M 9m7s
 1600K .......... .......... .......... .......... .......... 0% 2.52M 9m22s
 1650K .......... .......... .......... .......... .......... 0% 9.40M 9m13s
 1700K .......... .......... .......... .......... .......... 0% 3.92M 9m16s
 1750K .......... .......... .......... .......... .......... 0% 6.79M 9m11s
 1800K .......... .......... .......... .......... .......... 0% 4.27M 9m12s
 1850K .......... .......... .......... .......... .......... 0% 7.80M 9m6s
 1900K .......... .......... .......... .......... .......... 0% 12.1M 8m58s
 1950K .......... .......... .......... .......... .......... 0% 3.29M 9m4s
 2000K .......... .......... .......... .......... .......... 0% 9.66M 8m57s
 2050K .......... .......... .......... .......... .......... 0% 4.20M 8m59s
 2100K .......... .......... .......... .......... .......... 0% 5.45M 8m57s
 2150K .......... .......... .......... .......... .......... 0% 4.57M 8m58s
 2200K .......... .......... .......... .......... .......... 0% 7.09M 8m54s
 2250K .......... .......... .......... .......... .......... 0% 10.4M 8m48s
 2300K ......

In [0]:
%sh 
echo 'Iniciando download do Censo Escolar 2019...'
wget https://download.inep.gov.br/microdados/microdados_educacao_basica_2019.zip -O /dbfs/mnt/projetom03/censo_escolar_2019.zip

Iniciando download do Censo Escolar 2019...
--2021-03-25 14:40:04-- https://download.inep.gov.br/microdados/microdados_educacao_basica_2019.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2804740605 (2.6G) [application/zip]
Saving to: ‘/dbfs/mnt/projetom03/censo_escolar_2019.zip’

 0K .......... .......... .......... .......... .......... 0% 1.19M 37m33s
 50K .......... .......... .......... .......... .......... 0% 1.91M 30m28s
 100K .......... .......... .......... .......... .......... 0% 2.07M 27m29s
 150K .......... .......... .......... .......... .......... 0% 2.58M 24m56s
 200K .......... .......... .......... .......... .......... 0% 12.6M 20m39s
 250K .......... .......... .......... .......... .......... 0% 4.07M 19m2s
 300K .......... .......... .......... .......... .......... 0% 2.30M 19m5s
 350K .......... .......... .......... .......... .......... 0% 13.4M 17m7s
 400K .......... .......... .......... .......... .......... 0% 16.2M 15m31s
 450K .......... .......... .......... .......... .......... 0% 3.42M 15m16s
 500K .......... .......... .......... .......... .......... 0% 3.75M 14m58s
 550K .......... .......... .......... .......... .......... 0% 6.19M 14m19s
 600K .......... .......... .......... .......... .......... 0% 4.11M 14m3s
 650K .......... .......... .......... .......... .......... 0% 6.45M 13m32s
 700K .......... .......... .......... .......... .......... 0% 7.18M 13m3s
 750K .......... .......... .......... .......... .......... 0% 3.80M 12m58s
 800K .......... .......... .......... .......... .......... 0% 6.30M 12m37s
 850K .......... .......... .......... .......... .......... 0% 5.38M 12m23s
 900K .......... .......... .......... .......... .......... 0% 4.53M 12m15s
 950K .......... .......... .......... .......... .......... 0% 5.55M 12m2s
 1000K .......... .......... .......... .......... .......... 0% 6.01M 11m49s
 1050K .......... .......... .......... .......... .......... 0% 6.89M 11m34s
 1100K .......... .......... .......... .......... .......... 0% 4.41M 11m30s
 1150K .......... .......... .......... .......... .......... 0% 6.08M 11m20s
 1200K .......... .......... .......... .......... .......... 0% 5.34M 11m13s
 1250K .......... .......... .......... .......... .......... 0% 4.05M 11m12s
 1300K .......... .......... .......... .......... .......... 0% 6.50M 11m3s
 1350K .......... .......... .......... .......... .......... 0% 3.88M 11m3s
 1400K .......... .......... .......... .......... .......... 0% 14.5M 10m47s
 1450K .......... .......... .......... .......... .......... 0% 5.73M 10m41s
 1500K .......... .......... .......... .......... .......... 0% 4.08M 10m41s
 1550K .......... .......... .......... .......... .......... 0% 6.80M 10m34s
 1600K .......... .......... .......... .......... .......... 0% 3.88M 10m35s
 1650K .......... .......... .......... .......... .......... 0% 6.55M 10m29s
 1700K .......... .......... .......... .......... .......... 0% 3.94M 10m30s
 1750K .......... .......... .......... .......... .......... 0% 13.5M 10m18s
 1800K .......... .......... .......... .......... .......... 0% 3.76M 10m20s
 1850K .......... .......... .......... .......... .......... 0% 7.21M 10m14s
 1900K .......... .......... .......... .......... .......... 0% 4.12M 10m15s
 1950K .......... .......... .......... .......... .......... 0% 3.49M 10m19s
 2000K .......... .......... .......... .......... .......... 0% 8.63M 10m11s
 2050K .......... .......... .......... .......... .......... 0% 5.03M 10m9s
 2100K .......... .......... .......... .......... .......... 0% 11.2M 10m1s
 2150K .......... .......... .......... .......... .......... 0% 3.60M 10m4s
 2200K .......... .......... .......... .......... .......... 0% 8.45M 9m57s
 2250K .......... .......... .......... .......... .......... 0% 4.30M

In [0]:
dbutils.fs.cp("wasbs://m03container@m03storage.blob.core.windows.net/censo_escolar_2019.zip", "file:/tmp/censo_escolar_2019.zip")
dbutils.fs.cp("wasbs://m03container@m03storage.blob.core.windows.net/censo_escolar_2018.zip", "file:/tmp/censo_escolar_2018.zip")
dbutils.fs.cp("wasbs://m03container@m03storage.blob.core.windows.net/censo_escolar_2017.zip", "file:/tmp/censo_escolar_2017.zip")

Out[3]: True

Descompactação dos arquivos **.zip** sem a sua estrutura de diretórios em pastas para cada ano do censo

In [0]:
%sh
unzip -j /tmp/censo_escolar_2019.zip -d /tmp/censo2019/
unzip -j /tmp/censo_escolar_2018.zip -d /tmp/censo2018/
unzip -j /tmp/censo_escolar_2017.zip -d /tmp/censo2017/
unzip -j /tmp/censo2018/'*.zip' -d /tmp/censo2018/
unzip -j /tmp/censo2017/'*.zip' -d /tmp/censo2017/

Archive: /tmp/censo_escolar_2019.zip
 inflating: /tmp/censo2019/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx 
 inflating: /tmp/censo2019/Tabelas Auxiliares.xlsx 
 inflating: /tmp/censo2019/Cadastro_Aluno.pdf 
 inflating: /tmp/censo2019/Cadastro_Escola.pdf 
 inflating: /tmp/censo2019/Cadastro_Gestor_Escolar.pdf 
 inflating: /tmp/censo2019/Cadastro_Profissional_Escolar.pdf 
 inflating: /tmp/censo2019/Cadastro_Turma.pdf 
 inflating: /tmp/censo2019/DOCENTES_CO.CSV 
 inflating: /tmp/censo2019/DOCENTES_NORDESTE.CSV 
 inflating: /tmp/censo2019/DOCENTES_NORTE.CSV 
 inflating: /tmp/censo2019/DOCENTES_SUDESTE.CSV 
 inflating: /tmp/censo2019/DOCENTES_SUL.CSV 
 inflating: /tmp/censo2019/ESCOLAS.CSV 
 inflating: /tmp/censo2019/GESTOR.CSV 
 inflating: /tmp/censo2019/MATRICULA_CO.CSV 
 inflating: /tmp/censo2019/MATRICULA_NORDESTE.CSV 
 inflating: /tmp/censo2019/MATRICULA_NORTE.CSV 
 inflating: /tmp/censo2019/MATRICULA_SUDESTE.CSV 
 inflating: /tmp/censo2019/MATRICULA_SUL.CSV 
 inflating: /tmp/censo2019/MD5_microdados_ed_basica_2019.txt 
 inflating: /tmp/censo2019/TURMAS.CSV 
 inflating: /tmp/censo2019/Filtros Censo Ed Basica 2019.pdf 
 inflating: /tmp/censo2019/Leia-me 2019.pdf 
 inflating: /tmp/censo2019/Nota Informativa nз 01.pdf 
 inflating: /tmp/censo2019/Nota Informativa nз 02.pdf 
Archive: /tmp/censo_escolar_2018.zip
 inflating: /tmp/censo2018/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx 
 inflating: /tmp/censo2018/Tabelas Auxiliares.xlsx 
 inflating: /tmp/censo2018/Cadastro_Aluno.pdf 
 inflating: /tmp/censo2018/Cadastro_Escola.pdf 
 inflating: /tmp/censo2018/Cadastro_Profissional_Escolar.pdf 
 inflating: /tmp/censo2018/Cadastro_Turma.pdf 
 extracting: /tmp/censo2018/DOCENTES_CO.zip 
 extracting: /tmp/censo2018/DOCENTES_NORDESTE.zip 
 inflating: /tmp/censo2018/DOCENTES_NORTE.zip 
 inflating: /tmp/censo2018/DOCENTES_SUDESTE.zip 
 inflating: /tmp/censo2018/DOCENTES_SUL.zip 
 extracting: /tmp/censo2018/ESCOLAS.zip 
 extracting: /tmp/censo2018/MATRICULA_CO.zip 
 extracting: /tmp/censo2018/MATRICULA_NORDESTE.zip 
 extracting: /tmp/censo2018/MATRICULA_NORTE.zip 
 extracting: /tmp/censo2018/MATRICULA_SUDESTE.zip 
 extracting: /tmp/censo2018/MATRICULA_SUL.zip 
 inflating: /tmp/censo2018/md5_microdados_ed_basica_2018.txt 
 extracting: /tmp/censo2018/TURMAS.zip 
 inflating: /tmp/censo2018/Filtros da EducaЗ╞o Bаsica.pdf 
 inflating: /tmp/censo2018/Leia-me.pdf 
 inflating: /tmp/censo2018/Nota_informativa_versao_01.pdf 
Archive: /tmp/censo_escolar_2017.zip
 inflating: /tmp/censo2017/Dicionаrio de Dados da EducaЗ╞o Bаsica 2017.xlsx 
 inflating: /tmp/censo2017/Tabelas Auxiliares.xlsx 
 inflating: /tmp/censo2017/cadastro_aluno.pdf 
 inflating: /tmp/censo2017/cadastro_escola.pdf 
 inflating: /tmp/censo2017/cadastro_profissional_escolar.pdf 
 inflating: /tmp/censo2017/cadastro_turma.pdf 
 extracting: /tmp/censo2017/DOCENTES_CO.zip 
 extracting: /tmp/censo2017/DOCENTES_NORDESTE.zip 
 inflating: /tmp/censo2017/DOCENTES_NORTE.zip 
 extracting: /tmp/censo2017/DOCENTES_SUDESTE.zip 
 extracting: /tmp/censo2017/DOCENTES_SUL.zip 
 extracting: /tmp/censo2017/ESCOLAS.zip 
 extracting: /tmp/censo2017/MATRICULA_CO.zip 
 extracting: /tmp/censo2017/MATRICULA_NORDESTE.zip 
 extracting: /tmp/censo2017/MATRICULA_NORTE.zip 
 extracting: /tmp/censo2017/MATRICULA_SUDESTE.zip 
 extracting: /tmp/censo2017/MATRICULA_SUL.zip 
 inflating: /tmp/censo2017/md5_microdados_educaЗ╞o_bаsica_2017.txt 
 extracting: /tmp/censo2017/TURMAS.zip 
 inflating: /tmp/censo2017/Filtros Censo Ed Basica 2017.pdf 
 inflating: /tmp/censo2017/Leia-me 2017.pdf 
Archive: /tmp/censo2018/MATRICULA_SUDESTE.zip
 inflating: /tmp/censo2018/MATRICULA_SUDESTE.CSV 

Archive: /tmp/censo2018/MATRICULA_NORDESTE.zip
 inflating: /tmp/censo2018/MATRICULA_NORDESTE.CSV 

Archive: /tmp/censo2018/DOCENTES_NORDESTE.zip
 inflating: /tmp/censo2018/DOCENTES_NORDESTE.CSV 

Archive: /tmp/censo2018/TURMAS.zip
 inflating: /tmp/censo2018/TURMAS.CSV 

Archive: /tmp/censo2018/DOCENTES_NORTE.zip
 inflating: /tmp/censo2018/DOCENTES_NORT

Remoção de todos os arquivos que não possuem a extensão **.CSV**

In [0]:
%sh
shopt -s extglob
rm -v /tmp/censo2019/!(*.CSV)
rm -v /tmp/censo2018/!(*.CSV)
rm -v /tmp/censo2017/!(*.CSV)
shopt -u extglob

removed '/tmp/censo2019/Cadastro_Aluno.pdf'
removed '/tmp/censo2019/Cadastro_Escola.pdf'
removed '/tmp/censo2019/Cadastro_Gestor_Escolar.pdf'
removed '/tmp/censo2019/Cadastro_Profissional_Escolar.pdf'
removed '/tmp/censo2019/Cadastro_Turma.pdf'
removed '/tmp/censo2019/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx'
removed '/tmp/censo2019/Filtros Censo Ed Basica 2019.pdf'
removed '/tmp/censo2019/Leia-me 2019.pdf'
removed '/tmp/censo2019/MD5_microdados_ed_basica_2019.txt'
removed '/tmp/censo2019/Nota Informativa nз 01.pdf'
removed '/tmp/censo2019/Nota Informativa nз 02.pdf'
removed '/tmp/censo2019/Tabelas Auxiliares.xlsx'
removed '/tmp/censo2018/Cadastro_Aluno.pdf'
removed '/tmp/censo2018/Cadastro_Escola.pdf'
removed '/tmp/censo2018/Cadastro_Profissional_Escolar.pdf'
removed '/tmp/censo2018/Cadastro_Turma.pdf'
removed '/tmp/censo2018/Dicionаrio de Dados da EducaЗ╞o Bаsica.xlsx'
removed '/tmp/censo2018/DOCENTES_CO.zip'
removed '/tmp/censo2018/DOCENTES_NORDESTE.zip'
removed '/tmp/censo2018/DOCENTES_NORTE.zip'
removed '/tmp/censo2018/DOCENTES_SUDESTE.zip'
removed '/tmp/censo2018/DOCENTES_SUL.zip'
removed '/tmp/censo2018/ESCOLAS.zip'
removed '/tmp/censo2018/Filtros da EducaЗ╞o Bаsica.pdf'
removed '/tmp/censo2018/Leia-me.pdf'
removed '/tmp/censo2018/MATRICULA_CO.zip'
removed '/tmp/censo2018/MATRICULA_NORDESTE.zip'
removed '/tmp/censo2018/MATRICULA_NORTE.zip'
removed '/tmp/censo2018/MATRICULA_SUDESTE.zip'
removed '/tmp/censo2018/MATRICULA_SUL.zip'
removed '/tmp/censo2018/md5_microdados_ed_basica_2018.txt'
removed '/tmp/censo2018/Nota_informativa_versao_01.pdf'
removed '/tmp/censo2018/Tabelas Auxiliares.xlsx'
removed '/tmp/censo2018/TURMAS.zip'
removed '/tmp/censo2017/cadastro_aluno.pdf'
removed '/tmp/censo2017/cadastro_escola.pdf'
removed '/tmp/censo2017/cadastro_profissional_escolar.pdf'
removed '/tmp/censo2017/cadastro_turma.pdf'
removed '/tmp/censo2017/Dicionаrio de Dados da EducaЗ╞o Bаsica 2017.xlsx'
removed '/tmp/censo2017/DOCENTES_CO.zip'
removed '/tmp/censo2017/DOCENTES_NORDESTE.zip'
removed '/tmp/censo2017/DOCENTES_NORTE.zip'
removed '/tmp/censo2017/DOCENTES_SUDESTE.zip'
removed '/tmp/censo2017/DOCENTES_SUL.zip'
removed '/tmp/censo2017/ESCOLAS.zip'
removed '/tmp/censo2017/Filtros Censo Ed Basica 2017.pdf'
removed '/tmp/censo2017/Leia-me 2017.pdf'
removed '/tmp/censo2017/MATRICULA_CO.zip'
removed '/tmp/censo2017/MATRICULA_NORDESTE.zip'
removed '/tmp/censo2017/MATRICULA_NORTE.zip'
removed '/tmp/censo2017/MATRICULA_SUDESTE.zip'
removed '/tmp/censo2017/MATRICULA_SUL.zip'
removed '/tmp/censo2017/md5_microdados_educaЗ╞o_bаsica_2017.txt'
removed '/tmp/censo2017/Tabelas Auxiliares.xlsx'
removed '/tmp/censo2017/TURMAS.zip'

## Censo Escolar

### 1. Matrículas

#### 1.1. Ingestão dos dados no Delta Lake

In [0]:
# Leitura dos arquivos .CSV de cada ano e região, armazenando-os em dataframes separados.

matricula_co_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/MATRICULA_CO.CSV'))
matricula_norte_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/MATRICULA_NORTE.CSV'))
matricula_nordeste_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/MATRICULA_NORDESTE.CSV'))
matricula_sul_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/MATRICULA_SUL.CSV'))
matricula_sudeste_2017_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2017/MATRICULA_SUDESTE.CSV'))

matricula_co_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/MATRICULA_CO.CSV'))
matricula_norte_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/MATRICULA_NORTE.CSV'))
matricula_nordeste_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/MATRICULA_NORDESTE.CSV'))
matricula_sul_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/MATRICULA_SUL.CSV'))
matricula_sudeste_2018_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2018/MATRICULA_SUDESTE.CSV'))

matricula_co_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/MATRICULA_CO.CSV'))
matricula_norte_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/MATRICULA_NORTE.CSV'))
matricula_nordeste_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/MATRICULA_NORDESTE.CSV'))
matricula_sul_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/MATRICULA_SUL.CSV'))
matricula_sudeste_2019_bronze = (spark.read.option("sep", "|").option("header", "true").csv('file:/tmp/censo2019/MATRICULA_SUDESTE.CSV'))

In [0]:
# Armazenamento dos dataframes na camada Bronze do Delta Lake

matricula_co_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_co_2017')
matricula_norte_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_norte_2017')
matricula_nordeste_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_nordeste_2017')
matricula_sul_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_sul_2017')
matricula_sudeste_2017_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_sudeste_2017')

matricula_co_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_co_2018')
matricula_norte_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_norte_2018')
matricula_nordeste_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_nordeste_2018')
matricula_sul_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_sul_2018')
matricula_sudeste_2018_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_sudeste_2018')

matricula_co_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_co_2019')
matricula_norte_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_norte_2019')
matricula_nordeste_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_nordeste_2019')
matricula_sul_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_sul_2019')
matricula_sudeste_2019_bronze.write.format('delta').mode('overwrite').save(blobPath + bronzePath + 'matricula_sudeste_2019')

#### 1.2. Transformação dos dados

In [0]:
# Leitura dos dataframes salvos na camada Bronze

matricula_co_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_co_2017')
matricula_norte_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_norte_2017') 
matricula_nordeste_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_nordeste_2017') 
matricula_sul_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_sul_2017') 
matricula_sudeste_2017_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_sudeste_2017')

matricula_co_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_co_2018')
matricula_norte_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_norte_2018') 
matricula_nordeste_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_nordeste_2018') 
matricula_sul_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_sul_2018') 
matricula_sudeste_2018_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_sudeste_2018')

matricula_co_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_co_2019')
matricula_norte_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_norte_2019') 
matricula_nordeste_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_nordeste_2019') 
matricula_sul_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_sul_2019') 
matricula_sudeste_2019_silver = spark.read.format("delta").load(blobPath + bronzePath + 'matricula_sudeste_2019')

In [0]:
# Lista de colunas a serem selecionadas do dataframe original porém com novo título

renamedFields = ['NU_ANO_CENSO as Ano', 
                 'NU_IDADE_REFERENCIA as Idade_Referencia', 
                 'NU_IDADE as Idade',
                 'TP_SEXO as Sexo',
                 'TP_COR_RACA as Cor_Raca', 
                 'TP_NACIONALIDADE as Nacionalidade', 
                 'TP_ZONA_RESIDENCIAL as Zona_Residencial', 
                 'IN_NECESSIDADE_ESPECIAL as Necessidade_Especial', 
                 'TP_ETAPA_ENSINO as Etapa_Ensino', 
                 'IN_ESPECIAL_EXCLUSIVA as Especial_Exclusiva', 
                 'IN_REGULAR as Ensino_Regular', 
                 'IN_EJA as Ensino_EJA', 
                 'IN_PROFISSIONALIZANTE as Ensino_Profissionalizante', 
                 'TP_MEDIACAO_DIDATICO_PEDAGO as Mediacao_Didatico_Pedago', 
                 'TP_DEPENDENCIA as Dependencia_Escola', 
                 'TP_LOCALIZACAO as Localizacao_Escola']

In [0]:
# Alteração dos dataframes carregados com base na lista definida anteriormente

matricula_co_2017_silver = matricula_co_2017_silver.selectExpr(renamedFields)
matricula_norte_2017_silver = matricula_norte_2017_silver.selectExpr(renamedFields)
matricula_nordeste_2017_silver = matricula_nordeste_2017_silver.selectExpr(renamedFields)
matricula_sul_2017_silver = matricula_sul_2017_silver.selectExpr(renamedFields)
matricula_sudeste_2017_silver = matricula_sudeste_2017_silver.selectExpr(renamedFields)

matricula_co_2018_silver = matricula_co_2018_silver.selectExpr(renamedFields)
matricula_norte_2018_silver = matricula_norte_2018_silver.selectExpr(renamedFields)
matricula_nordeste_2018_silver = matricula_nordeste_2018_silver.selectExpr(renamedFields)
matricula_sul_2018_silver = matricula_sul_2018_silver.selectExpr(renamedFields)
matricula_sudeste_2018_silver = matricula_sudeste_2018_silver.selectExpr(renamedFields)

matricula_co_2019_silver = matricula_co_2019_silver.selectExpr(renamedFields)
matricula_norte_2019_silver = matricula_norte_2019_silver.selectExpr(renamedFields)
matricula_nordeste_2019_silver = matricula_nordeste_2019_silver.selectExpr(renamedFields)
matricula_sul_2019_silver = matricula_sul_2019_silver.selectExpr(renamedFields)
matricula_sudeste_2019_silver = matricula_sudeste_2019_silver.selectExpr(renamedFields)

In [0]:
# Unificação dos dataframes das regiões em um único dataframe para cada ano e armazenando na camada Prata

matricula_co_2017_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'matricula_2017')
matricula_norte_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2017')
matricula_nordeste_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2017')
matricula_sul_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2017')
matricula_sudeste_2017_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2017')

matricula_co_2018_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'matricula_2018')
matricula_norte_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2018')
matricula_nordeste_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2018')
matricula_sul_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2018')
matricula_sudeste_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2018')

matricula_co_2019_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'matricula_2019')
matricula_norte_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2019')
matricula_nordeste_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2019')
matricula_sul_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2019')
matricula_sudeste_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matricula_2019')

In [0]:
# Leitura dos dataframes unificados

matricula_2017_silver = spark.read.format("delta").load(blobPath + silverPath + 'matricula_2017')
matricula_2018_silver = spark.read.format("delta").load(blobPath + silverPath + 'matricula_2018')
matricula_2019_silver = spark.read.format("delta").load(blobPath + silverPath + 'matricula_2019')

In [0]:
# Validação da unificação dos dataframes, comparando o total de registros

totalBronzeRows = matricula_co_2017_silver.count() + matricula_norte_2017_silver.count() + matricula_nordeste_2017_silver.count() + matricula_sul_2017_silver.count() + matricula_sudeste_2017_silver.count()

print("2017:", matricula_2017_silver.count() == totalBronzeRows)

totalBronzeRows = matricula_co_2018_silver.count() + matricula_norte_2018_silver.count() + matricula_nordeste_2018_silver.count() + matricula_sul_2018_silver.count() + matricula_sudeste_2018_silver.count()

print("2018:", matricula_2018_silver.count() == totalBronzeRows)

totalBronzeRows = matricula_co_2019_silver.count() + matricula_norte_2019_silver.count() + matricula_nordeste_2019_silver.count() + matricula_sul_2019_silver.count() + matricula_sudeste_2019_silver.count()

print("2019:", matricula_2019_silver.count() == totalBronzeRows)

2017: True
2018: True
2019: True

In [0]:
# Verificando a quantidade de registros null por coluna de cada dataframe

display(matricula_2017_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in matricula_2017_silver.columns]))
display(matricula_2018_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in matricula_2018_silver.columns]))
display(matricula_2019_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in matricula_2019_silver.columns]))

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
0,0,0,0,0,0,0,0,5292576,5292576,5292576,5292576,5292576,0,0,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
0,0,0,0,0,0,0,0,3373546,3373546,3373546,3373546,3373546,0,0,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
0,0,0,0,0,0,56,0,3292477,3292477,3292477,3292477,3292477,0,0,0


In [0]:
#Definição de dicionário para substituição dos valores nulos no formato -> 'Coluna':'Novo Valor'

nullSubstDict = {'Zona_Residencial': '0', 'Etapa_Ensino': '0', 'Especial_Exclusiva': '2', 'Ensino_Regular': '2', 'Ensino_EJA': '2', 'Ensino_Profissionalizante': '2'}

In [0]:
# Substituição dos valores nulos de cada dataframe com base no dicionário definido

matricula_2017_silver = matricula_2017_silver.fillna(nullSubstDict)
matricula_2018_silver = matricula_2018_silver.fillna(nullSubstDict)
matricula_2019_silver = matricula_2019_silver.fillna(nullSubstDict)

In [0]:
# Verificando a quantidade de registros null por coluna de cada dataframe

display(matricula_2017_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in matricula_2017_silver.columns]))
display(matricula_2018_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in matricula_2018_silver.columns]))
display(matricula_2019_silver.select([count(when(col(c).isNull(), c)).alias(c) for c in matricula_2019_silver.columns]))

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Unificando os 3 dataframes em um único dataframe

matricula_2017_silver.write.format('delta').mode('overwrite').save(blobPath + silverPath + 'matriculas')
matricula_2018_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matriculas')
matricula_2019_silver.write.format('delta').mode('append').save(blobPath + silverPath + 'matriculas')

In [0]:
# Leitura do dataframe final para execução das últimas transformações

matricula_silver = spark.read.format("delta").load(blobPath + silverPath + 'matriculas')

In [0]:
display(matricula_silver)

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
2017,12,13,1,3,1,1,0,21,0,1,0,0,1,2,1
2017,10,10,2,1,1,1,1,15,0,1,0,0,1,3,1
2017,11,12,2,1,1,2,0,20,0,1,0,0,1,4,1
2017,29,29,2,1,1,1,0,69,0,0,1,0,1,3,1
2017,13,13,2,3,1,1,0,19,0,1,0,0,1,3,1
2017,11,11,1,3,1,1,0,19,0,1,0,0,1,3,1
2017,11,12,2,3,1,1,0,19,0,1,0,0,1,4,1
2017,15,15,2,1,1,1,0,26,0,1,0,0,1,4,1
2017,11,11,1,0,1,1,0,19,0,1,0,0,1,4,1
2017,30,31,2,0,1,2,0,69,0,0,1,0,1,3,1


In [0]:
# Alteração dos valores de cada coluna de acordo com as informações correspondentes

matricula_silver = matricula_silver.replace(['1', '2'], 
                                            ['Masculino', 'Feminino'], 'Sexo')

matricula_silver = matricula_silver.replace(['0', '1', '2', '3', '4', '5'], 
                                            ['Não Declarada', 'Branca', 'Preta', 'Parda', 'Amarela', 'Indígena'], 'Cor_Raca')

matricula_silver = matricula_silver.replace(['1', '2', '3'], 
                                            ['Brasileira', 'Brasileira - nascido no exterior ou naturalizado', 'Estrangeira'], 'Nacionalidade')

matricula_silver = matricula_silver.replace(['0', '1', '2'], 
                                            ['Não Informado', 'Urbana', 'Rural'], 'Zona_Residencial')

matricula_silver = matricula_silver.replace(['0', '1'], 
                                            ['Não', 'Sim'], 'Necessidade_Especial')

matricula_silver = matricula_silver.replace(['1', '2', '3'], 
                                            ['Presencial', 'Semipresencial', 'Educação a Distância - EAD'], 'Mediacao_Didatico_Pedago')

matricula_silver = matricula_silver.replace(['0', '1', '2'], 
                                            ['Não', 'Sim', 'Não Informado'], 'Especial_Exclusiva')

matricula_silver = matricula_silver.replace(['0', '1', '2'], 
                                            ['Não', 'Sim', 'Não Informado'], 'Ensino_Regular')

matricula_silver = matricula_silver.replace(['0', '1', '2'], 
                                            ['Não', 'Sim', 'Não Informado'], 'Ensino_EJA')

matricula_silver = matricula_silver.replace(['0', '1', '2'], 
                                            ['Não', 'Sim', 'Não Informado'], 'Ensino_Profissionalizante')

matricula_silver = matricula_silver.replace(['1', '2', '3', '4'], 
                                            ['Federal', 'Estadual', 'Municipal', 'Privada'], 'Dependencia_Escola')

matricula_silver = matricula_silver.replace(['1', '2'], 
                                            ['Urbana', 'Rural'], 'Localizacao_Escola')

In [0]:
# Devido ao tamanho dos dados da coluna 'Etapa_Ensino' é definido duas variáveis para os valores para melhor visualização

etapaOriginais = ['0', '1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '14', '15', '16', '17', '18', '19', '20', '21', '41', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '68', '65', '67', '69', '70', '71', '72', '73', '74']

etapaNovos = [
  'Não Informado',
  'Educação Infantil - Creche',
  'Educação Infantil - Pré-escola',
  'Ensino Fundamental de 8 anos - 1ª Série',
  'Ensino Fundamental de 8 anos - 2ª Série',
  'Ensino Fundamental de 8 anos - 3ª Série',
  'Ensino Fundamental de 8 anos - 4ª Série',
  'Ensino Fundamental de 8 anos - 5ª Série',
  'Ensino Fundamental de 8 anos - 6ª Série',
  'Ensino Fundamental de 8 anos - 7ª Série',
  'Ensino Fundamental de 8 anos - 8ª Série',
  'Ensino Fundamental de 9 anos - 1º Ano',
  'Ensino Fundamental de 9 anos - 2º Ano',
  'Ensino Fundamental de 9 anos - 3º Ano',
  'Ensino Fundamental de 9 anos - 4º Ano',
  'Ensino Fundamental de 9 anos - 5º Ano',
  'Ensino Fundamental de 9 anos - 6º Ano',
  'Ensino Fundamental de 9 anos - 7º Ano',
  'Ensino Fundamental de 9 anos - 8º Ano',
  'Ensino Fundamental de 9 anos - 9º Ano',
  'Ensino Médio - 1ª Série',
  'Ensino Médio - 2ª Série',
  'Ensino Médio - 3ª Série',
  'Ensino Médio - 4ª Série',
  'Ensino Médio - Não Seriada',
  'Curso Técnico Integrado (Ensino Médio Integrado) 1ª Série',
  'Curso Técnico Integrado (Ensino Médio Integrado) 2ª Série',
  'Curso Técnico Integrado (Ensino Médio Integrado) 3ª Série',
  'Curso Técnico Integrado (Ensino Médio Integrado) 4ª Série',
  'Curso Técnico Integrado (Ensino Médio Integrado) Não Seriada',
  'Ensino Médio - Normal/Magistério 1ª Série',
  'Ensino Médio - Normal/Magistério 2ª Série',
  'Ensino Médio - Normal/Magistério 3ª Série',
  'Ensino Médio - Normal/Magistério 4ª Série',
  'Curso Técnico - Concomitante',
  'Curso Técnico - Subsequente',
  'Curso FIC Concomitante',
  'EJA - Ensino Fundamental - Projovem Urbano',
  'Curso FIC integrado na modalidade EJA - Nivel Médio',
  'EJA - Ensino Fundamental - Anos Iniciais',
  'EJA - Ensino Fundamental - Anos Finais',
  'EJA - Ensino Médio',
  'EJA - Ensino Fundamental - Anos Iniciais e Anos Finais',
  'Curso FIC Integrado EJA - Nivel Fundamental',
  'Curso FIC Integrado EJA - Nivel Médio'
]

In [0]:
# Alteração dos valores da coluna 'Etapa_Ensino'

matricula_silver = matricula_silver.replace(etapaOriginais, etapaNovos, 'Etapa_Ensino')

In [0]:
# Visualização do dataframe após alteração

display(matricula_silver)

Ano,Idade_Referencia,Idade,Sexo,Cor_Raca,Nacionalidade,Zona_Residencial,Necessidade_Especial,Etapa_Ensino,Especial_Exclusiva,Ensino_Regular,Ensino_EJA,Ensino_Profissionalizante,Mediacao_Didatico_Pedago,Dependencia_Escola,Localizacao_Escola
2017,12,13,Masculino,Parda,Brasileira,Urbana,Não,Ensino Fundamental de 9 anos - 8º Ano,Não,Sim,Não,Não,Presencial,Estadual,Urbana
2017,10,10,Feminino,Branca,Brasileira,Urbana,Sim,Ensino Fundamental de 9 anos - 2º Ano,Não,Sim,Não,Não,Presencial,Municipal,Urbana
2017,11,12,Feminino,Branca,Brasileira,Rural,Não,Ensino Fundamental de 9 anos - 7º Ano,Não,Sim,Não,Não,Presencial,Privada,Urbana
2017,29,29,Feminino,Branca,Brasileira,Urbana,Não,EJA - Ensino Fundamental - Anos Iniciais,Não,Não,Sim,Não,Presencial,Municipal,Urbana
2017,13,13,Feminino,Parda,Brasileira,Urbana,Não,Ensino Fundamental de 9 anos - 6º Ano,Não,Sim,Não,Não,Presencial,Municipal,Urbana
2017,11,11,Masculino,Parda,Brasileira,Urbana,Não,Ensino Fundamental de 9 anos - 6º Ano,Não,Sim,Não,Não,Presencial,Municipal,Urbana
2017,11,12,Feminino,Parda,Brasileira,Urbana,Não,Ensino Fundamental de 9 anos - 6º Ano,Não,Sim,Não,Não,Presencial,Privada,Urbana
2017,15,15,Feminino,Branca,Brasileira,Urbana,Não,Ensino Médio - 2ª Série,Não,Sim,Não,Não,Presencial,Privada,Urbana
2017,11,11,Masculino,Não Declarada,Brasileira,Urbana,Não,Ensino Fundamental de 9 anos - 6º Ano,Não,Sim,Não,Não,Presencial,Privada,Urbana
2017,30,31,Feminino,Não Declarada,Brasileira,Rural,Não,EJA - Ensino Fundamental - Anos Iniciais,Não,Não,Sim,Não,Presencial,Municipal,Urbana


In [0]:
matricula_silver.write.format('delta').mode('overwrite').save(blobPath + goldPath + 'matriculas')
matricula_silver.write.format('delta').mode('overwrite').saveAsTable('Ouro_Censo_Escolar_Matriculas')